## **Retrieval Augmented Generation (RAG)**

- 대규모 언어 모델(LLM)에 외부 지식을 연결하여 더 정확하고 최신의 정보를 제공하는 AI 프레임워크

- RAG의 작동 원리
    - 사용자 질문 → 관련 문서 검색 → 컨텍스트와 함께 LLM에 전달 → 답변 생성

- RAG vs 일반 LLM 비교

    | 구분 | 일반 LLM | RAG |
    |------|----------|-----|
    | 정보 소스 | 사전 훈련 데이터만 | 외부 지식베이스 + 사전 훈련 데이터 |
    | 최신성 | 훈련 시점까지 | 실시간 업데이트 가능 |
    | 정확성 | 환각(hallucination) 가능성 | 검증된 문서 기반 답변 |
    | 사용 사례 | 일반적인 질문 답변 | 특정 도메인의 전문적 답변 |

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
# .env 파일 생성
from dotenv import load_dotenv
load_dotenv()

# OpenAI API 키 설정 (필요시)
# OPENAI_API_KEY=your_openai_api_key_here

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob
from pprint import pprint
import json
from pathlib import Path

---

## 문서 로더 (Document Loaders)

- **Document Loader**는 다양한 소스에서 문서를 로드하여 LangChain의 `Document` 객체로 변환하는 도구

- **Document 객체 구조**

    ```python
    from langchain_core.documents import Document

    # Document 객체의 기본 구조
    document = Document(
        page_content="문서의 텍스트 내용",
        metadata={
            "source": "문서 출처",
            "page": 1,
            "title": "문서 제목"
        }
    )
    ```

- **문서 로더의 종류**
    - PDF 파일 로더
    - 웹 페이지 로더 
    - CSV 데이터 로더
    - 디렉토리 로더
    - HTML 데이터 로더
    - JSON 데이터 로더
    - Markdown 데이터 로더
    - Microsoft Office 데이터 로더

### **PDF 파일 로더**

- uv add langchain_community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('data/labor_law.pdf', mode='page')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')
print()

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")
    print("-" * 50)

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

In [ ]:
pprint(pdf_docs[0].metadata)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('data/labor_law.pdf', mode='single')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

In [ ]:
pprint(pdf_docs[0].metadata)

#### 📝 연습 문제 1: PDF 파일 로드

In [ ]:
"""
📌 실습: data 폴더에 있는 다음 파일을 로드하세요:
1. PDF 파일을 페이지별로 분할하여 로드 (data/personal_info_law.pdf)
2. 각 페이지의 메타데이터 확인
3. 전체 문서를 단일 Document로 로드
"""

# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_community.document_loaders import PyPDFLoader

# 1. 페이지별 분할 로드
pdf_loader_pages = PyPDFLoader('data/personal_info_law.pdf', mode='page')
pages_docs = pdf_loader_pages.load()

print(f"페이지별 분할 시 문서 개수: {len(pages_docs)}")
print(f"첫 페이지 메타데이터: {pages_docs[0].metadata}")
print(f"첫 페이지 내용 (100자): {pages_docs[0].page_content[:100]}")

# 2. 단일 문서로 로드
pdf_loader_single = PyPDFLoader('data/personal_info_law.pdf', mode='single')
single_doc = pdf_loader_single.load()

print(f"\n단일 문서 로드 시 문서 개수: {len(single_doc)}")
print(f"전체 문서 길이: {len(single_doc[0].page_content)} 문자")
```
</details>


---

## 텍스트 분할 (Chunking)

-  🎯 텍스트 분할이 필요한 이유
    1. **토큰 제한**: LLM은 입력 토큰 수에 제한이 있음
    2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과 제공
    3. **메모리 효율성**: 대용량 문서의 효율적 처리

- 📊 분할 전략 비교

    | 방법 | 장점 | 단점 | 사용 사례 |
    |------|------|------|----------|
    | CharacterTextSplitter | 단순, 빠름 | 문맥 고려 안함 | 간단한 텍스트 |
    | RecursiveCharacterTextSplitter | 문맥 보존 우수 | 계산 복잡 | 일반적인 문서 |
    | TokenTextSplitter | 정확한 토큰 수 | 토크나이저 의존 | API 비용 최적화 |
    | SemanticChunker | 의미 기반 분할 | 느림, 비용 많음 | 플롯, 시나리오 등 |

In [ ]:
print(f'첫 번째 문서: {pdf_docs[0]}')

In [ ]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

### 1. **CharacterTextSplitter**
- 가장 기본적인 분할 방식
- 문자 수를 기준으로 텍스트를 분할
- 단순하지만 문맥을 고려하지 않는다는 단점이 있음

- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할 - split_text() 메서드 사용
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")

In [ ]:
# 첫 번째 청크의 텍스트 출력
print(f'첫 번째 청크의 텍스트: {chunks[0]}')

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 문장 구분자를 개행문자로 설정
text_splitter = CharacterTextSplitter(
    separator = "\n",        # 청크 구분자: 개행문자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])   # 첫 번째 문서만 분할

# 분할된 청크 개수 출력
print(f'분할된 청크 개수: {len(chunks)}')

In [ ]:
# 각 청크의 텍스트 길이 출력
for i, chunk in enumerate(chunks):
    print(f'청크 {i+1}의 텍스트 길이: {len(chunk.page_content)}')

In [ ]:
# 첫 번째 청크의 텍스트 출력
print(f'첫 번째 청크의 텍스트: {chunks[0].page_content}')

### 2. **RecursiveCharacterTextSplitter**

- 재귀적으로 텍스트를 분할
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 분할
- 문맥을 더 잘 보존할 수 있음  


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 기본 재귀 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # 우선순위 순서
)

# Document 객체 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 각 청크의 길이 확인
for i, chunk in enumerate(chunks[:10]):
    print(f"청크 {i+1}: {len(chunk.page_content)} 문자")

### 3. **정규표현식 사용**

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = CharacterTextSplitter(
    separator=r'(?<=[.!?])\s+',  # 각 Document 객체의 page_content 속성을 문장으로 분할
    chunk_size=1000,
    chunk_overlap=200,
    is_separator_regex=True,      # 구분자가 정규식인지 여부: True
    keep_separator=True,          # 구분자 유지 여부: True
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)  # 모든 문서를 분할
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("...")
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

In [ ]:
# 법률 문서와 공문서에 적합한 문장 분할기 생성
# 문장 끝 후 공백 문자가 있는 패턴을 찾아서 분할
regex_pattern = r'(?<=[.!?])\s+'

sentence_splitter = CharacterTextSplitter(
    separator=regex_pattern,
    chunk_size=1000,
    chunk_overlap=200,
    is_separator_regex=True,
    keep_separator=True
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = sentence_splitter.split_documents(pdf_docs)  # 모든 문서를 분할
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("...")
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

### 4. **토큰 수를 기반으로 분할**

`(1) tiktoken`  

- OpenAI에서 만든 BPE Tokenizer

- 언어 모델 토크나이저 비교

    | 모델 | 토크나이저 | 어휘 크기 | 특징 |
    |------|-----------|----------|------|
    | GPT-4.1 | o200k_base | 200,000 | 최신 모델, 다국어 효율성 개선 |
    | GPT-4o | o200k_base | 200,000 | 멀티모달 지원 |
    | o1, o3, o4 시리즈 | o200k_base | 200,000 | 최신 추론 모델 |
    | GPT-4 | cl100k_base | 100,000 | 이전 세대 플래그십 모델 |
    | GPT-3.5-Turbo | cl100k_base | 100,000 | 효율적인 대화 모델 |
    | Codex | p50k_base | 50,000 | 코딩 전용 모델 |


- 임베딩 모델 토크나이저 비교

    | 모델명 | 토크나이저 | 어휘 크기 | 임베딩 차원 | 특징 |
    |------|-----------|----------|------|------|
    | text-embedding-3-large | cl100k_base | 100,000 | 3,072 | 최고 성능|
    | text-embedding-3-small | cl100k_base | 100,000 | 1,536 | 고효율 |
    | text-embedding-ada-002 | cl100k_base | 100,000 | 1,536 | 레거시 모델|


In [ ]:
# 첫번째 문서 객체의 텍스트 길이
len(pdf_docs[0].page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])  # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(chunk.page_content[:50])
    print("-" * 50)
    print(chunk.page_content[-50:])
    print("=" * 50)
    print()

In [ ]:
import tiktoken

# TikToken 인코더 초기화
tiktoken_tokenizer = tiktoken.get_encoding("cl100k_base")

# 각 청크에 대해 토큰화 및 단어 수 확인
for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tiktoken_tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = [tiktoken_tokenizer.decode([token]) for token in tokens[:10]]
    print(token_strings)

    print("=" * 50)
    print()

`(2) Hugging Face 토크나이저`  
- Hugging Face tokenizer 모델의 토큰 수를 기준으로 분할
- uv add langchain_huggingface sentence_transformers

- 오픈소스 언어 모델 토크나이저 

    | 모델 패밀리 | 모델명 | 토크나이저 | 어휘 크기 | 알고리즘 | 특징 |
    |------------|--------|-----------|----------|----------------|------|
    | **Meta LLaMA** | LLaMA 1, LLaMA 2 | SentencePiece | 32,000 | BPE | SentencePiece BPE with byte-fallback |
    | | LLaMA 3 | tiktoken-style | 128,000 | Byte-level BPE | Non-SentencePiece, 직접 바이트 레벨 BPE |
    | **XLM-RoBERTa** | XLM-RoBERTa-base | SentencePiece | 250,000 | BPE/Unigram | 100+ 언어 지원, 대용량 어휘 |
    | | XLM-RoBERTa-large | SentencePiece | 250,000 | BPE/Unigram | 다국어 최적화 |
    | **BERT** | BERT-base/large (uncased) | WordPiece | 30,522 | WordPiece | 소문자 변환 |
    | | BERT-base/large (cased) | WordPiece | 28,996 | WordPiece | 대소문자 유지 |
    | | BERT-multilingual | WordPiece | 119,547 | WordPiece | 104개 언어 지원 |

- 오픈소스 임베딩 모델 토크나이저 

    | 모델명 | 토크나이저 | 어휘 크기 | 최대 입력 길이 | 베이스 모델 | 특징 |
    |--------|-----------|----------|--------------|------------|------|
    | BGE-M3 | SentencePiece | 250,000 | 8,192 tokens | XLM-RoBERTa-large | Multi-Functionality (dense/sparse/multi-vector), Multi-Linguality (100+ 언어), Multi-Granularity |
    | BGE-large/base/small | BERT WordPiece | 30,522 | 512 tokens | BERT | 영어/중국어 최적화 |
    | BGE-reranker-v2-m3 | SentencePiece | 250,000 | 512 tokens | XLM-RoBERTa | 경량 크로스-인코더 재순위화 |

In [ ]:
from transformers import AutoTokenizer 

# Huggingface Tokenizer 초기화
bge_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

bge_tokenizer

In [ ]:
# 토크나이저 인코딩 - 문장을 토큰(ID)으로 변환
tokens = bge_tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

In [ ]:
# 토큰을 출력 (토큰 ID를 실제 토큰(문자열)로 변환)
print(bge_tokenizer.convert_ids_to_tokens(tokens)) 

In [ ]:
# 디코딩 - 토큰을 문자열로 변환
print(bge_tokenizer.decode(tokens, skip_special_tokens=True))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Huggingface 토크나이저를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=bge_tokenizer,
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]]) # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = bge_tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = bge_tokenizer.convert_ids_to_tokens(tokens[:10]) 
    print(token_strings)

    print("=" * 50)
    print()

#### 📝 연습 문제 2: 최적 청크 크기 찾기

In [ ]:
"""
📌 실습: 다양한 청크 크기로 분할하고 비교하세요 (data/personal_info_law.pdf)
- 청크 크기: 500, 1000, 1500
- 오버랩: 청크 크기의 20%
- 각 설정에서 생성된 청크 수와 평균 길이 계산
"""

# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 문서 로드 (이전 코드 활용)
pdf_loader = PyPDFLoader('data/personal_info_law.pdf', mode='single')
docs = pdf_loader.load()

# 다양한 청크 크기 테스트
chunk_sizes = [500, 1000, 1500]
results = []

for size in chunk_sizes:
    overlap = int(size * 0.2)  # 20% 오버랩
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap,
        length_function=len,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    chunks = splitter.split_documents(docs)
    avg_length = sum(len(chunk.page_content) for chunk in chunks) / len(chunks)
    
    results.append({
        'chunk_size': size,
        'overlap': overlap,
        'num_chunks': len(chunks),
        'avg_length': avg_length
    })
    
    print(f"청크 크기 {size}:")
    print(f"  - 청크 수: {len(chunks)}")
    print(f"  - 평균 길이: {avg_length:.2f}")
    print(f"  - 오버랩: {overlap}")
    print("-" * 50)
```
</details>


---

## 문서 임베딩 (Document Embedding)

- 🎯 임베딩이란?
    - 텍스트를 벡터(숫자 배열)로 변환하는 과정
    - 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환 

- 목적:
    - 텍스트 간 유사도 계산 가능
    - 벡터 데이터베이스 저장 및 검색
    - 의미 기반 문서 검색 구현

- 📊 임베딩 모델 비교

    | 모델 | 차원 | 언어 지원 | 비용 | 성능 | 사용 사례 |
    |------|------|----------|------|------|----------|
    | OpenAI text-embedding-3-small | 1536 | 다국어 | 유료 | 높음 | 프로덕션 |
    | OpenAI text-embedding-3-large | 3072 | 다국어 | 유료 | 최고 | 고성능 요구 |
    | BAAI/bge-m3 | 1024 | 다국어 | 무료 | 높음 | 한국어 특화 |
    | sentence-transformers/all-MiniLM-L6-v2 | 384 | 영어 | 무료 | 중간 | 로컬 개발 |

### 1. OpenAI 임베딩

`(1) 기본 설정`

In [ ]:
from langchain_openai import OpenAIEmbeddings

# 기본 임베딩 모델
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

print(f"컨텍스트 길이: {embeddings_openai.embedding_ctx_length}")

`(2) 문서 임베딩`

In [ ]:
# 문서 컬렉션 임베딩
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "컴퓨터 비전은 이미지를 분석하는 기술입니다."
]

# 배치 임베딩 (효율적)
openai_small_embeddings = embeddings_openai.embed_documents(documents)
print(f"임베딩 벡터 수: {len(openai_small_embeddings)}")
print(f"각 벡터 차원: {len(openai_small_embeddings[0])}")

In [ ]:
# 쿼리 임베딩
query = "AI 기술에 대해 알려주세요"
query_embedding = embeddings_openai.embed_query(query)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")

### 2. Hugging Face 임베딩

`(1) BGE-M3 모델 (한국어 우수)`

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# BGE-M3 모델 (다국어, 한국어 성능 우수)
embeddings_bge = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},        # 'cuda' for GPU
    encode_kwargs={'normalize_embeddings': True}  # L2 정규화
)

# BGE-M3 모델로 문서 임베딩
bge_hf_embeddings = embeddings_bge.embed_documents(documents)
print(f"한국어 임베딩 차원: {len(bge_hf_embeddings[0])}")

`(2) 경량 모델`

In [ ]:
# 빠른 처리를 위한 경량 모델
embedding_gte = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-multilingual-base",
    model_kwargs={'device': 'cpu', 'trust_remote_code': True},  
    encode_kwargs={'normalize_embeddings': True}
)
    
# 경량 모델로 문서 임베딩
alibaba_hf_embeddings = embedding_gte.embed_documents(documents)
print(f"경량 모델 한국어 임베딩 차원: {len(alibaba_hf_embeddings[0])}") 

### 3. Ollama 임베딩 (로컬)

- 설치: uv add langchain_ollama

In [ ]:
from langchain_ollama import OllamaEmbeddings

# Ollama 서버가 실행 중이어야 함
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3",                    # 사용할 모델
    # base_url="http://localhost:11434"  # Ollama 서버 주소
)

# 로컬 임베딩
local_embeddings = embeddings_ollama.embed_documents(documents)

print(f"로컬 임베딩 벡터 수: {len(local_embeddings)}")
print(f"각 벡터 차원: {len(local_embeddings[0])}")

### 4. 유사도 계산 및 검색


`(1) 코사인 유사도`

In [ ]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

def find_most_similar(query, doc_embeddings, documents, embeddings_model):
    """가장 유사한 문서 찾기"""
    # 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 가장 유사한 문서 인덱스
    most_similar_idx = np.argmax(similarities)
    
    return {
        "document": documents[most_similar_idx],
        "similarity": similarities[most_similar_idx],
        "index": most_similar_idx
    }

In [ ]:
# 쿼리와 문서 임베딩을 사용하여 가장 유사한 문서 찾기
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# OpenAIEmbeddings 사용한 유사도 검색 (OpenAI)
for query in queries:
    result = find_most_similar(query, openai_small_embeddings, documents, embeddings_openai)

    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("-" * 50)

In [ ]:
# HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
for query in queries:
    result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)

    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("-" * 50)

In [ ]:
# Alibaba-NLP/gte-multilingual-base 모델로 유사도 검색

for query in queries:
    result = find_most_similar(query, alibaba_hf_embeddings, documents, embedding_gte)

    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("-" * 50)


In [ ]:
# Ollama 모델로 유사도 검색 (bge-m3)

for query in queries:
    result = find_most_similar(query, local_embeddings, documents, embeddings_ollama)

    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("-" * 50)
    

#### 📝 연습 문제 3: 임베딩 모델 성능 비교

In [ ]:
"""
📌 실습: 서로 다른 임베딩 모델의 성능 비교 
- BGE-M3와 GTE-multilingual 모델 비교
- 동일한 쿼리에 대한 검색 결과 비교
- 처리 시간 측정
"""

# 테스트 문서와 쿼리
test_docs = [
    "의료 인공지능은 질병 진단에 활용됩니다.",
    "딥러닝은 의료 영상 분석에 효과적입니다.",
    "환자 데이터 보안은 매우 중요합니다.",
    "원격 진료는 의료 접근성을 향상시킵니다.",
    "생명정보학은 유전체 데이터를 분석합니다."
]

test_query = "AI를 활용한 질병 진단 방법"

# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_huggingface import HuggingFaceEmbeddings
import time
import numpy as np
from langchain_community.utils.math import cosine_similarity

# 테스트 문서와 쿼리
test_docs = [
    "의료 인공지능은 질병 진단에 활용됩니다.",
    "딥러닝은 의료 영상 분석에 효과적입니다.",
    "환자 데이터 보안은 매우 중요합니다.",
    "원격 진료는 의료 접근성을 향상시킵니다.",
    "생명정보학은 유전체 데이터를 분석합니다."
]

test_query = "AI를 활용한 질병 진단 방법"

# 두 가지 임베딩 모델 초기화
embedding_models = {
    "BGE-M3": HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    ),
    "GTE-multilingual": HuggingFaceEmbeddings(
        model_name="Alibaba-NLP/gte-multilingual-base",
        model_kwargs={'device': 'cpu', 'trust_remote_code': True},
        encode_kwargs={'normalize_embeddings': True}
    )
}

# 모델별 성능 비교
for model_name, embedding_model in embedding_models.items():
    print(f"\n=== {model_name} 모델 ===")
    
    # 임베딩 시간 측정
    start_time = time.time()
    doc_embeddings = embedding_model.embed_documents(test_docs)
    query_embedding = embedding_model.embed_query(test_query)
    embed_time = time.time() - start_time
    
    # 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 결과 정렬
    ranked_results = sorted(
        zip(test_docs, similarities), 
        key=lambda x: x[1], 
        reverse=True
    )
    
    print(f"임베딩 시간: {embed_time:.4f}초")
    print(f"임베딩 차원: {len(doc_embeddings[0])}")
    print("\n검색 결과 (상위 3개):")
    for i, (doc, score) in enumerate(ranked_results[:3], 1):
        print(f"{i}. (유사도: {score:.4f}) {doc}")
```

</details>


---

## 벡터 저장소 (Vector Store)

- 벡터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
- 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
- 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능 

- 주요 기능:
    - 벡터 색인화: 효율적인 검색을 위한 데이터 구조화를 수행
    - 근접 이웃 검색: 주어진 쿼리와 가장 유사한 벡터들을 검색 
    - 메타데이터 관리: 벡터와 관련된 부가 정보를 함께 저장하고 검색

- 사용 사례:
    - 시맨틱 문서 검색: 문서의 의미를 이해하여 검색
    - 추천 시스템: 유사한 아이템을 추천
    - 중복 데이터 감지: 유사한 콘텐츠를 검색 
    - 질의응답 시스템: 관련 문서에서 답변을 생성하는데 필요한 근거를 검색 


- 📊 벡터 저장소 비교

    | 종류 | 장점 | 단점 | 사용 사례 |
    |------|------|------|----------|
    | Chroma | 설치 간단, 로컬 친화적 | 대용량 처리 한계 | 개발, 프로토타입 |
    | FAISS | 매우 빠름, 확장성 우수 | 설정 복잡 | 대용량 검색 |
    | Pinecone | 완전 관리형, 고성능 | 유료, 클라우드 의존 | 프로덕션 |
    | Weaviate | GraphQL 지원, 하이브리드 검색 | 학습 곡선 | 복합 검색 |

### 1. **Chroma**

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
- `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화 (근로기준법 문서)
pdf_loader = PyPDFLoader('data/labor_law.pdf', mode='single')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화 (토큰 수 기준 분할)
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=bge_tokenizer,
    chunk_size=500, 
    chunk_overlap=100,
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs) 

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(f"토큰 개수: {len(bge_tokenizer.encode(chunk.page_content))}")
    print(f"{chunk.page_content[:100]}...{chunk.page_content[-100:]}")
    print("="*100)

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings   

# Ollama 임베딩 모델 생성
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3"
)

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(  
    documents=chunks,
    embedding=embeddings_ollama,    # 임베딩 사용
    collection_name="labor_law_rag",    # 컬렉션 이름
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(2) 벡터 저장소 로드`  

In [ ]:
from langchain_chroma import Chroma

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="labor_law_rag",
    embedding_function=embeddings_ollama,
    persist_directory="./chroma_db",
)

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(3) 문서 검색`  
- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=5는 상위 5개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "탄력 근로에 대해서 설명해주세요"
results = chroma_db.similarity_search(
    query,
    k=5,
)

print("유사도 검색 결과:")
for doc in results:
    print(f"{doc.page_content}")
    print("=" * 100)

- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [ ]:
query = "탄력 근로에 대해서 설명해주세요"
results = chroma_db.similarity_search_with_score(
    query,
    k=5,
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f" 메타데이터: {doc.metadata}")
    print("=" * 100)

### 2. **FAISS**

- 고성능의 유사도 검색을 위한 라이브러리
- `faiss-cpu` 패키지 설치

`(1) 벡터 저장소 초기화`

- uv add faiss-cpu

In [ ]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터 저장소 생성
faiss_db = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings_openai,  # OpenAI 임베딩 사용
)

# 로컬 저장
faiss_db.save_local("./faiss_index")

In [ ]:
# 저장된 문서의 개수
print(f"FAISS 벡터 저장소에 저장된 문서 개수: {faiss_db.index.ntotal}")

`(2) 벡터 저장소 로드`  

In [ ]:
# 로드
faiss_db = FAISS.load_local(
    "./faiss_index",
    embeddings_openai,
    allow_dangerous_deserialization=True
)

# 문서 개수 확인
print(f"로드된 FAISS 벡터 저장소의 문서 개수: {faiss_db.index.ntotal}")

In [ ]:
# 검색
similar_docs = faiss_db.similarity_search(query, k=5)

print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

---

## 검색기 (Retriever)

- 벡터 저장소를 기반으로 사용자 질의에 가장 관련성 높은 문서를 검색하는 LangChain 인터페이스

- 📊 검색 전략 비교

    | 전략 | 설명 | 장점 | 단점 | 사용 사례 |
    |------|------|------|------|----------|
    | similarity | 단순 유사도 검색 | 빠름, 직관적 | 다양성 부족 | 일반적인 검색 |
    | similarity_score_threshold | 임계값 기반 검색 | 품질 보장 | 결과 수 불안정 | 고품질 결과 필요 |
    | mmr | 최대 한계 관련성 | 다양성 우수 | 느림 | 포괄적 정보 필요 |



### 1. 기본 유사도 검색 (similarity)

In [ ]:
# 벡터 저장소를 Retriever로 변환
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 결과
)

# 검색 실행
query = "탄력 근로에 대해 설명해주세요"
retrieved_docs = retriever.invoke(query)

print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content[:200]}...")
    print(f"출처: {doc.metadata.get('source', 'Unknown')}")
    print("-" * 50)

### 2. 임계값 기반 검색 (similarity_score_threshold)

In [ ]:
# 유사도 점수 임계값 기반 검색
threshold_retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.6,  # 0.6 이상의 유사도만
        "k": 10                   # 최대 10개까지
    }
)

retrieved_docs = threshold_retriever.invoke(query)

print("유사도 점수 임계값 기반 검색 결과:")
print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print("-" * 50)

In [ ]:
# 실제 유사도 점수 확인
from langchain_community.utils.math import cosine_similarity

for i, doc in enumerate(retrieved_docs):
    # 실제 유사도 계산
    doc_embedding = embeddings_ollama.embed_query(doc.page_content)
    query_embedding = embeddings_ollama.embed_query(query)
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    print(f"문서 {i+1} (유사도: {similarity:.4f}):")
    print(f"{doc.page_content[:100]}...")
    print("-" * 50)

### 3. MMR (Maximal Marginal Relevance) 검색

In [ ]:
# MMR 검색 - 관련성과 다양성의 균형
mmr_retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,                # 최종 반환할 문서 수
        "fetch_k": 20,         # 초기 후보 문서 수
        "lambda_mult": 0.5     # 관련성 vs 다양성 (0=최대 다양성, 1=최대 관련성)
    }
)

mmr_docs = mmr_retriever.invoke(query)

print(f"MMR 검색 결과 수: {len(mmr_docs)}")

for i, doc in enumerate(mmr_docs):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print("-" * 50)

#### 📝 연습 문제 4: Chroma 벡터 저장소 구축

In [ ]:
"""
📌 실습: 의료 문서로 벡터 저장소 구축 (data/personal_info_law.pdf)
1. PDF 문서 로드 및 분할
2. Chroma 벡터 저장소 생성
3. 메타데이터 필터링 검색 구현
"""

# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import hashlib

# 1. 문서 준비
pdf_loader = PyPDFLoader('data/personal_info_law.pdf', mode='page')
docs = pdf_loader.load()

# 2. 텍스트 분할 (메타데이터 추가)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(docs)

# 3. 벡터 저장소 생성
embeddings = OllamaEmbeddings(model="bge-m3")

vectorstore = Chroma.from_documents(
    documents=chunks,  
    embedding=embeddings,
    collection_name="personal_info_law_rag",
    persist_directory="./chroma_practice"
)

print(f"저장된 문서 수: {vectorstore._collection.count()}")

# 4. 메타데이터 필터링 검색
# 특정 페이지에서만 검색
results = vectorstore.similarity_search(
    "인공지능 의료",
    k=3,
    filter={"page": 0}  # 첫 페이지에서만 검색
)

print("\n첫 페이지에서 검색한 결과:")
for doc in results:
    print(f"- 페이지 {doc.metadata.get('page')}: {doc.page_content[:100]}...")
```

</details>

---

##  Naive RAG 구현 

- 검색(Retrieval)과 생성(Generation)을 연결하여 외부 지식을 기반으로 답변을 생성하는 파이프라인

- 🔄 RAG 워크플로우
    - 사용자 질문 → 관련 문서 검색 → 컨텍스트 구성 → LLM 프롬프트 → 답변 생성

`(1) 벡터 저장소 로드`
- `Chroma` 벡터 저장소를 로드

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# Ollama 임베딩 모델 생성
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3",  # 사용할 모델 이름
    )

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="labor_law_rag",
    embedding_function=embeddings_ollama,
    persist_directory="./chroma_db",
)

# 문서 개수 확인
print(f"저장된 문서 수: {chroma_db._collection.count()}")

`(2) 검색기(Retriever) 초기화`

- mmr 검색을 사용하는 Retriever 사용
- 다양성을 높이는 설정을 사용

In [ ]:
# mmr 검색기 생성
retriever = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 5,                  # 검색할 문서의 수
        'fetch_k': 10,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,      # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)

In [ ]:
# 검색 테스트 
query = "탄력 근로에 대해 설명해주세요"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-" * 100)

`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 기본 RAG 프롬프트 템플릿
basic_template = """주어진 컨텍스트를 기반으로 질문에 답변하세요.

<컨텍스트>
{context}
</컨텍스트>

<질문>
{question}
</질문>
"""

basic_prompt = ChatPromptTemplate.from_template(basic_template)

# 템플릿 출력
basic_prompt.pretty_print()

In [ ]:
# 커스텀 프롬프트 템플릿
advanced_template = """당신은 전문적인 문서 분석 AI입니다. 주어진 컨텍스트를 바탕으로 정확하고 유용한 답변을 제공하세요.

<답변 지침>
- 컨텍스트에 있는 정보만을 사용하여 답변하세요
- 확실하지 않은 정보는 "명확하지 않습니다"라고 명시하세요
- 답변은 논리적이고 구조화된 형태로 제공하세요
- 가능한 경우 구체적인 예시나 수치를 포함하세요
</답변 지침>

<답변 형식>
## **핵심 답변:** (질문에 대한 직접적인 답변)
## **세부 설명:** (추가적인 설명이나 배경 정보)
## **관련 정보:** (컨텍스트에서 발견된 연관 정보)
</답변 형식>

<컨텍스트>
{context}
</컨텍스트>

<질문>
{question}
</질문>
"""

advanced_prompt = ChatPromptTemplate.from_template(advanced_template)

# 템플릿 출력
advanced_prompt.pretty_print()

`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOllama 사용 ('qwen3:4b' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성 

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.7,
    top_p=0.9,
)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
rag_chain = (
    RunnableParallel(
        {
            "context": retriever | RunnableLambda(format_docs), 
            "question": RunnablePassthrough()
        }
    )
    | advanced_prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "탄력 근로에 대해서 설명해주세요"
output = rag_chain.invoke(query)

In [ ]:
print(output)

---

# 🎯 실습: RAG 체인 구성

- RAG 체인을 구성하여 질문에 대한 답변을 생성해보세요.
- 문서 로딩, 임베딩, 벡터 저장소 구축, 검색기 설정 등을 포함하여 전체 RAG 파이프라인을 구현합니다.
- 대상 문서: data/personal_info_law.pdf

In [ ]:
# 여기에 코드를 작성하세요